In [22]:
import lime
import sklearn
import warnings
import itertools
import numpy as np
import pandas as pd
import lime.lime_tabular
import googleapiclient.discovery

from google.cloud import storage

In [23]:
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [24]:
storage_client = storage.Client(project='energy-forecasting')
bucket = storage_client.get_bucket('energyforecast')
blob = bucket.blob('data/csv/MLDataTrain.csv')
train =  blob.download_as_string(storage_client).split('\n')
blob = bucket.blob('data/csv/MLDataTest.csv')
test =  blob.download_as_string(storage_client).split('\n')

In [25]:
names = [
  'price',
  'date_utc',
  'day',
  'hour',
  'prev_week_min',
  'prev_week_25th',
  'prev_week_50th',
  'prev_week_75th',
  'prev_week_max'
]
w_names = [
  'temperature',
  'wind_speed_100m',
  'wind_direction_100m',
  'air_density',
  'precipitation',
  'wind_gust',
  'radiation',
  'wind_speed',
  'wind_direction',
  'pressure'
]

w_names = [ ['loc'+str(i)+"_"+n for n in w_names] for i in range(18)]

w_names = list(itertools.chain.from_iterable(w_names))

feature_names = names + w_names 
categorical_features = [2, 3]

In [27]:
def convert_to_matrix(data):
    new_lst = []
    for string in data[1:]:
        new_lst.append(string.split(','))

    df = pd.DataFrame.from_records(new_lst)
    df = df.dropna()
    df.head()

    le = sklearn.preprocessing.LabelEncoder()
    df.iloc[:, 1] = le.fit_transform(df.iloc[:, 1])

    return (df.values.astype(float), le)

In [28]:
train, le = convert_to_matrix(train)
test, _ = convert_to_matrix(test)

In [29]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    train,
    feature_names=feature_names,
    class_names=['energy'],
    categorical_features=categorical_features,
    verbose=True,
    mode='regression',
    )

In [30]:
def predict(project, model, record, version=None):
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': record}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [31]:
def refine(x):
    x = x.split(',')
    x[2] = str(int(float(x[2])))
    x[3] = str(int(float(x[3])))
    return ','.join(x)

def transform_row(x):
    x = x.astype(str)
    x[1] = le.inverse_transform([int(float(x[1]))])[0]
    tmp = ','.join(list(x))
    refiend_row = refine(tmp)
    return refiend_row

In [32]:
def lime_predict(rows):
    if len(rows.shape) > 1:
        predictions = []
        for row in rows:
            prediction = predict('energy-forecasting',
                                 'energyforecaster',
                                 transform_row(row), 'new_energy'
                                 )[0]['predictions']
            predictions.extend(prediction)
    else:
        predictions = predict('energy-forecasting', 'energyforecaster',
                              transform_row(rows), 'new_energy'
                              )[0]['predictions']
    return np.array(predictions)

In [35]:
i = 105
exp = explainer.explain_instance(test[i], lime_predict, num_features=5, num_samples=10)

Intercept 0.497121336971721
Prediction_local [0.52496435]
Right: 0.531932532787323


In [38]:
exp.show_in_notebook(show_table=True)